In [4]:
import duckdb
con = duckdb.connect(database='sozmed.duckdb', read_only=True)

def quote(s):
  return f"'{s}'"

## Interactive / complex / 13

In [3]:
person1Id = 100
person2Id = 200

sql = f"""
WITH RECURSIVE
-- Bidirectional breadth first search
bfs(dir, node, depth) AS (
    SELECT dd,nn,ll FROM (VALUES (false, {person1Id}::bigint, 0), (true, {person2Id}::bigint, 0)) t(dd,nn,ll)
    UNION ALL
    (
        with
        rec AS (SELECT dir, node, depth FROM bfs),
        md AS (SELECT max(depth) AS d FROM rec),
        last AS (SELECT dir, node, depth FROM rec WHERE depth = (SELECT d FROM md)),
        new AS (
            SELECT dir, pkp.person2id AS dst, min(depth) + 1 AS md
            FROM last, person_knows_person pkp
            WHERE last.node = pkp.person1id AND pkp.person2id not in (SELECT node FROM rec WHERE last.dir = rec.dir)
            group by dir, pkp.person2id
        )
        SELECT dir, dst, md
        FROM
            (SELECT dir, dst, md FROM new
            union all
            SELECT dir, node, depth
            FROM rec) t
        WHERE true 
            AND EXISTS (SELECT * FROM new)
            AND NOT EXISTS (
                    SELECT *
                    FROM last r1, rec r2
                    WHERE r1.dir = true AND r2.dir = false AND r1.node = r2.node
                )

    )
),
md AS (SELECT max(depth) AS d FROM bfs),
found(depth) AS (
    SELECT min(r1.depth + r2.depth)
    FROM bfs r1, bfs r2
    WHERE r1.depth = (SELECT d FROM md) and
          r1.dir = true AND r2.dir = false AND r1.node = r2.node

)
SELECT coalesce((SELECT depth FROM found), -1)
;

"""
# print(sql)
con.execute(sql)

InvalidInputException: Invalid Input Error: Attempting to execute an unsuccessful or closed pending query result
Error: Not implemented Error: UNIONS are not supported in recursive CTEs yet

## BI Read 17

In [5]:
tag = quote("Slavoj_Žižek")
delta = 4

sql = f"""
WITH MyMessage as (
    SELECT *
    FROM Message
    WHERE Id in (SELECT Id FROM Message_hasTag_Tag WHERE TagId IN (SELECT id FROM Tag WHERE Tag.name = {tag}))
)
SELECT Message1.CreatorPersonId AS "person1.id", count(DISTINCT Message2.Id) AS messageCount
FROM MyMessage Message1
-- (message2 <date filtering>)
JOIN MyMessage Message2
 ON (Message1.creationDate + '{delta} hour'::interval) < Message2.creationDate
JOIN MyMessage Comment
 ON Comment.ParentMessageId = Message2.Id
JOIN Forum_hasMember_Person Forum_hasMember_Person2
  ON Forum_hasMember_Person2.ForumId = Message1.ContainerForumId -- forum1
 AND Forum_hasMember_Person2.PersonId = Comment.CreatorPersonId -- person2
JOIN Forum_hasMember_Person Forum_hasMember_Person3
  ON Forum_hasMember_Person3.ForumId = Message1.ContainerForumId -- forum1
 AND Forum_hasMember_Person3.PersonId = Message2.CreatorPersonId -- person3
WHERE Message1.ContainerForumId <> Message2.ContainerForumId
  -- person2 <> person3
  AND Forum_hasMember_Person2.PersonId <> Forum_hasMember_Person3.PersonId
  AND NOT EXISTS (SELECT 1
                  FROM Forum_hasMember_Person Forum_hasMember_Person1
                  WHERE Forum_hasMember_Person1.ForumId = Message2.ContainerForumId -- forum2
                    AND Forum_hasMember_Person1.PersonId = Message1.CreatorPersonId -- person1
                 )
GROUP BY Message1.CreatorPersonId
ORDER BY messageCount DESC, Message1.CreatorPersonId ASC
LIMIT 10
;
"""
# print(sql)
con.execute(sql)